## Motivation

Every two years, every member of the United States House of Representatives is up for election. After being elected, members of the House are given a set budget from the legislature itself to hire staff, buy office equipment, and defray other costs of legislating and addressing constituent concerns. While each office gets the same amount of money from Congress to spend on these purposes, congressional offices have discretion over how that allowance is actually spent, and we would like to see whether some spending patterns are associated with higher political success. 

# Part 1: Getting/Formatting the Data

For this project, we decided to use the [House Office Expenditure Data](https://www.propublica.org/datastore/dataset/house-office-expenditures) from ProPublica as it contains well formatted data about house expenditures from 2009 to 2018. The main downside of this dataset is that it is missing data from the most recent midterm election, but we still have almost 10 years of data to work with.

To programatically access the datasets we are working with, we have included copies here [repository](https://github.com/AndrewTrackim/cmsc320-final/raw/master/house-expenditure/).

In [35]:
import pandas as pd
# just finna test with one of the files to see what happens
frames = []
# #manually add stuff for 2009 since only Q3 and Q4 are present
# frames.append(pd.read_csv('https://github.com/AndrewTrackim/cmsc320-final/raw/master/house-expenditure/2009Q3-house-disburse-detail.csv').dropna(subset = ['BIOGUIDE_ID']))
# frames.append(pd.read_csv('https://github.com/AndrewTrackim/cmsc320-final/raw/master/house-expenditure/2009Q4-house-disburse-detail.csv').dropna(subset = ['BIOGUIDE_ID']))


#automate the dataframes from 2010 to 2017
for i in range(2016, 2018):
    for j in range(1, 5):
        df = pd.read_csv('https://github.com/AndrewTrackim/cmsc320-final/raw/master/house-expenditure/' + str(i) + 'Q' + str(j) +'-house-disburse-detail.csv', thousands=',')
        df.dropna(subset = ['BIOGUIDE_ID'], inplace=True)
        df["QUARTER"] = str(i) + 'Q' + str(j)#df.apply(lambda row: str(i) + 'Q' + str(j))
        frames.append(df)
        

# manually add stuff for 2018 since only Q1 is present
df = pd.read_csv('https://github.com/AndrewTrackim/cmsc320-final/raw/master/house-expenditure/2018Q1-house-disburse-detail.csv', thousands=',')
df.dropna(subset = ['BIOGUIDE_ID'], inplace=True)
df["QUARTER"] = str(2018) + 'Q' + str(1)
frames.append(df)

house_data = pd.concat(frames)
house_data

C:\Users\karth\Anaconda3\lib\site-packages\ipykernel_launcher.py:24: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



,AMOUNT,BIOGUIDE_ID,CATEGORY,DATE,END DATE,OFFICE,PAYEE,PROGRAM,PURPOSE,QUARTER,RECIP (orig.),RECORDID,SORT SEQUENCE,START DATE,TRANSCODE,TRANSCODELONG,YEAR
5387,-25.85,A000374,FRANKED MAIL,01-31,01/31/16,HON. RALPH ABRAHAM,NaN,NaN,FRANKED MAIL,2016Q1,NaN,FLG0055718,NaN,01/20/16,GL,General ledger,2016
5388,627.83,A000374,FRANKED MAIL,02-29,01/31/16,HON. RALPH ABRAHAM,UNITED STATES POSTAL SERVICE,NaN,FRANKED MAIL,2016Q1,UNITED STATES POSTAL SERVICE,00844090,NaN,01/03/16,AP,Accounts payable,2016
5389,-62.50,A000374,FRANKED MAIL,02-29,02/29/16,HON. RALPH ABRAHAM,NaN,NaN,FRANKED MAIL,2016Q1,NaN,FLG0056519,NaN,02/20/16,GL,General ledger,2016
5390,944.14,A000374,FRANKED MAIL,03-23,02/29/16,HON. RALPH ABRAHAM,UNITED STATES POSTAL SERVICE,NaN,FRANKED MAIL,2016Q1,UNITED STATES POSTAL SERVICE,00849298,NaN,02/01/16,AP,Accounts payable,2016
5391,-9.45,A000374,FRANKED MAIL,03-31,03/31/16,HON. RALPH ABRAHAM,NaN,NaN,FRANKED MAIL,2016Q1,NaN,FLG0057391,NaN,03/20/16,GL,General ledger,2016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61345,54.85,Z000017,SUPPLIES AND MATERIALS,2/19/18,12/30/17,2017 HON. LEE M. ZELDIN,CITI PCARD-READYREFRESH BY NESTLE,OFFICIAL EXPENSES OF MEMBERS,OFFICE SUPPLIES (OUTSIDE),2018Q1,NaN,974834,DETAIL,12/29/17,AP,NaN,2017
61346,110.00,Z000017,SUPPLIES AND MATERIALS,2/19/18,1/26/18,2017 HON. LEE M. ZELDIN,CITI PCARD-TIMES REVIEW NEWSPAP,OFFICIAL EXPENSES OF MEMBERS,PUBLICATIONS/REFERENCE MAT'L,2018Q1,NaN,974834,DETAIL,12/29/17,AP,NaN,2018
61347,2183.20,Z000017,SUPPLIES AND MATERIALS,NaN,NaN,2017 HON. LEE M. ZELDIN,NaN,OFFICIAL EXPENSES OF MEMBERS,SUPPLIES AND MATERIALS TOTALS:,2018Q1,NaN,NaN,SUBTOTAL,NaN,NaN,NaN,2018
61348,38303.93,Z000017,SUPPLIES AND MATERIALS,NaN,NaN,2017 HON. LEE M. ZELDIN,NaN,OFFICIAL EXPENSES OF MEMBERS,OFFICIAL EXPENSES OF MEMBERS TOTALS:,2018Q1,NaN,NaN,SUBTOTAL,NaN,NaN,NaN,2018


[election data](https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/IG0UN2)

In [36]:
election_data = pd.read_csv('https://github.com/AndrewTrackim/cmsc320-final/raw/master/house-expenditure/1976-2020-house.csv')

# filter out the years before 2009 and after 2018
election_data = election_data[election_data['year'] >= 2009]
election_data = election_data[election_data['year'] <= 2018]

election_data

,year,state,state_po,state_fips,state_cen,state_ic,office,district,stage,runoff,special,candidate,party,writein,mode,candidatevotes,totalvotes,unofficial,version,fusion_ticket
22553,2010,ALABAMA,AL,1,63,41,US HOUSE,1,GEN,NaN,False,DAVID WALTER,CONSTITUTION,False,TOTAL,26357,156281,False,20220331,False
22554,2010,ALABAMA,AL,1,63,41,US HOUSE,1,GEN,NaN,False,JO BONNER,REPUBLICAN,False,TOTAL,129063,156281,False,20220331,False
22555,2010,ALABAMA,AL,1,63,41,US HOUSE,1,GEN,NaN,False,WRITEIN,NaN,True,TOTAL,861,156281,False,20220331,False
22556,2010,ALABAMA,AL,1,63,41,US HOUSE,2,GEN,NaN,False,BOBBY BRIGHT,DEMOCRAT,False,TOTAL,106865,219028,False,20220331,False
22557,2010,ALABAMA,AL,1,63,41,US HOUSE,2,GEN,NaN,False,MARTHA ROBY,REPUBLICAN,False,TOTAL,111645,219028,False,20220331,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29631,2018,WYOMING,WY,56,83,68,US HOUSE,0,GEN,NaN,False,DANIEL CLYDE CUMMINGS,CONSTITUTION,False,TOTAL,6070,201245,False,20220331,False
29632,2018,WYOMING,WY,56,83,68,US HOUSE,0,GEN,NaN,False,GREG HUNTER,DEMOCRAT,False,TOTAL,59903,201245,False,20220331,False
29633,2018,WYOMING,WY,56,83,68,US HOUSE,0,GEN,NaN,False,LIZ CHENEY,REPUBLICAN,False,TOTAL,127963,201245,False,20220331,False
29634,2018,WYOMING,WY,56,83,68,US HOUSE,0,GEN,NaN,False,RICHARD BRUBAKER,LIBERTARIAN,False,TOTAL,6918,201245,False,20220331,False


Try to figure out what categories we are working with and how these categories should be group into broader categories

In [37]:
from collections import defaultdict
categories = defaultdict(lambda: 0)
def add_set(row):
      categories[row['CATEGORY']] += 1

#apply add_set to the house_data
house_data.apply(add_set, axis = 1);
categories

defaultdict(<function __main__.<lambda>()>,
            {'FRANKED MAIL': 25231,
             'PERSONNEL COMPENSATION': 97559,
             'TRAVEL': 177810,
             'RENT, COMMUNICATION, UTILITIES': 59560,
             'PRINTING AND REPRODUCTION': 33221,
             'OTHER SERVICES': 36402,
             'SUPPLIES AND MATERIALS': 134555,
             'EQUIPMENT': 24966,
             'TRANSPORTATION OF THINGS': 248,
             'PERSONNEL BENEFITS': 1,
             'RENT COMMUNICATION UTILITIES': 18459,
             'RENT  COMMUNICATION  UTILITIES': 92169,
             'BENEFITS TO FORMER PERSONNEL': 5})

In [38]:
replace = {
    'RENT  COMMUNICATION  UTILITIES': 'RENT, COMMUNICATION, UTILITIES',
    'RENT COMMUNICATION UTILITIES': 'RENT, COMMUNICATION, UTILITIES'
}
house_data.replace(to_replace=replace, value=None, inplace=True)
categories = defaultdict(lambda: 0)
def add_set(row):
      categories[row['CATEGORY']] += 1
      
house_data.apply(add_set, axis = 1);
categories

defaultdict(<function __main__.<lambda>()>,
            {'FRANKED MAIL': 25231,
             'PERSONNEL COMPENSATION': 97559,
             'TRAVEL': 177810,
             'RENT, COMMUNICATION, UTILITIES': 170188,
             'PRINTING AND REPRODUCTION': 33221,
             'OTHER SERVICES': 36402,
             'SUPPLIES AND MATERIALS': 134555,
             'EQUIPMENT': 24966,
             'TRANSPORTATION OF THINGS': 248,
             'PERSONNEL BENEFITS': 1,
             'BENEFITS TO FORMER PERSONNEL': 5})

In [39]:

hd_by_quarter = house_data.groupby('QUARTER')
hd_by_quarter.size()
hd_by_quarter.groups

{'2016Q1': Int64Index([ 5387,  5388,  5389,  5390,  5391,  5392,  5393,  5394,  5395,
              5396,
             ...
             87796, 87797, 87798, 87799, 87800, 87801, 87802, 87803, 87804,
             87805],
            dtype='int64', length=82406),
 '2016Q2': Int64Index([ 5518,  5519,  5520,  5521,  5522,  5523,  5524,  5525,  5526,
              5527,
             ...
             85008, 85009, 85010, 85011, 85012, 85013, 85014, 85015, 85016,
             85017],
            dtype='int64', length=79481),
 '2016Q3': Int64Index([ 5693,  5694,  5695,  5696,  5697,  5698,  5699,  5700,  5701,
              5702,
             ...
             78981, 78982, 78983, 78984, 78985, 78986, 78987, 78988, 78989,
             78990],
            dtype='int64', length=73270),
 '2016Q4': Int64Index([ 5493,  5494,  5495,  5496,  5497,  5498,  5499,  5500,  5501,
              5502,
             ...
             76883, 76884, 76885, 76886, 76887, 76888, 76889, 76890, 76891,
             76

In [56]:
quarters = house_data['QUARTER'].unique()
categories = house_data['CATEGORY'].unique()
for quarter in quarters[0:1]:
    print(quarter)
    hd_by_quarter = house_data[house_data['QUARTER'] == quarter]
    bio_ids = house_data['BIOGUIDE_ID'].unique()

    spending_per_candidate = hd_by_quarter.groupby(['BIOGUIDE_ID','CATEGORY'])['AMOUNT'].sum()
    for bio_id in bio_ids:
        print(spending_per_candidate[bio_id])
    # spending_per_candidate.to_csv('spending_per_candidate_' + quarter + '.csv')

2016Q1
CATEGORY
EQUIPMENT                            466.20
FRANKED MAIL                        2433.05
OTHER SERVICES                      9697.34
PERSONNEL COMPENSATION            208960.60
PRINTING AND REPRODUCTION           2255.50
RENT, COMMUNICATION, UTILITIES     19348.00
SUPPLIES AND MATERIALS              3770.78
TRAVEL                             20827.93
Name: AMOUNT, dtype: float64
CATEGORY
EQUIPMENT                           2163.69
FRANKED MAIL                       20807.28
OTHER SERVICES                      7139.75
PERSONNEL COMPENSATION            278020.63
PRINTING AND REPRODUCTION          31032.73
RENT, COMMUNICATION, UTILITIES     24069.64
SUPPLIES AND MATERIALS             11784.75
TRAVEL                             11188.31
Name: AMOUNT, dtype: float64
CATEGORY
EQUIPMENT                           1685.25
FRANKED MAIL                        2226.95
OTHER SERVICES                      5686.53
PERSONNEL COMPENSATION            238973.95
PRINTING AND REPRODUCTION   

KeyError: 'A000022'